In [1]:
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import package_dataprocess.Nilm_classes
import numpy as np
import keras
# from nilm_ukdale import preprocess
from keras import Model
from keras.models import Sequential
import os
import gc
import pandas as pd
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import PIL.ImageOps
from keras import Model
from keras import layers
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import MaxPooling3D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import GRU, TimeDistributed, Bidirectional
from keras.layers import Embedding
import matplotlib.pyplot as plt

一切都是命运石之门的选择！


In [2]:
#只要在这里改周数和每一周开始的unix时间戳即可
week = 'week9'
unixtime = 1456704000

In [3]:
#创建空numpy数组用来保存一会读到的数据
fridge_data = np.zeros(100800)
television_data = np.zeros(100800)
kettle_data = np.zeros(100800)
microwave_data = np.zeros(100800)
washerdryer_data = np.zeros(100800)

#下面这个将被用来保存的内容是：所有电器有功功率求和，来代替整间房子总有功功率
sum_data_num = np.zeros(100800)
allappliance = np.zeros((52,100800))

In [4]:
filenames = os.listdir('DATA\\UKData_by_hour\\'+week+'\\LowFreq')
dp = package_dataprocess.Nilm_classes.Nilm_dataprocess(path_csv="DATA\\UKData_by_hour\\"+week+"\\vicurve_name.xlsx",
                                                       path_img="DATA\\UKData_by_hour\\"+week+"\\vicurve32x32\\",
                                                       dimension = 52)
count = 0
for count,i in enumerate(filenames[:52]):
    temp = np.zeros(100800)
    name = i[11:-4]
    for time in range(7*24):
        name_data = np.load('DATA\\UKData_by_hour\\'+week+'\\LowFreq\\' + str(unixtime + 3600 * time) + '_' + name + '.npy')
        name_data = np.array(name_data)
        # print(name_data.shape)
        if len(name_data) > 600:
            name_data = name_data[:600]
        elif len(name_data) < 600:
            name_data = np.pad(name_data, (0, 600 - len(name_data)), 'constant', constant_values=(0, 0))
        name_data = np.where(name_data > 3, name_data, 0)  # 底噪3w
        allappliance[count][time * 600:(time + 1) * 600] =  name_data
        temp[time * 600:(time + 1) * 600] = name_data
        sum_data_num = sum_data_num + temp
        
#allappliance[0] = allappliance[0].reshape(allappliance[0].shape[0], 1)
print(allappliance.shape)
all = np.zeros((52,100800,1))
sum_label = np.zeros((100800,52)) 
for channel in range(52):
    for i in range(100800):
        all[channel][i][0]=allappliance[channel][i]
for channel in range(52):
    sum_label += dp.create_label(all[channel], channel, 100) #here，最后的15是有功功率阈值
sumlabel = sum_label.T

(52, 100800)


In [5]:
hhh = np.zeros(52)
names = filenames[:52]
for channel in range(52):
    for i in range(100800):
        if(sumlabel[channel][i]==1):
            hhh[channel]+=1
    print(names[channel][11:-4])#是否显示电器名
    print(hhh[channel]/100800)       

Active subwoofer0
9.92063492063492e-06
Audio amplifier0
9.92063492063492e-06
Audio system0
0.0
Baby monitor0
0.0
Boiler0
0.009831349206349206
Breadmaker0
0.0
Broadband router0
0.0
Charger0
0.0
Clothes iron0
0.0
Coffee maker0
0.001001984126984127
Computer monitor0
0.00034722222222222224
Computer0
0.0
Desktop computer0
0.036140873015873015
Dish washer0
0.024880952380952382
Ethernet switch0
0.0
Fan0
0.0
Fridge freezer0
0.025009920634920635
Hair dryer0
0.0
Hair straighteners0
0.0
HTPC0
0.0004365079365079365
Kettle0
0.005277777777777778
Laptop computer0
0.0
Light0
0.11673611111111111
Light1
0.0
Light10
1.984126984126984e-05
Light11
2.9761904761904762e-05
Light12
0.0
Light13
0.0
Light14
0.0
Light2
0.0
Light3
0.0
Light4
0.13663690476190476
Light5
0.0
Light6
0.0
Light7
0.0
Light8
0.0
Light9
0.0
Microwave0
0.002509920634920635
Mobile phone charger0
0.0
Mobile phone charger1
0.0
Oven0
0.0
Printer0
0.0002876984126984127
Radio0
0.0
Radio1
0.0
Solar thermal pumping station0
1.984126984126984e-05
So